In [1]:
import math

ROTATION_AMOUNTS = [
    7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22,
    5, 9, 14, 20, 5, 9, 14, 20, 5, 9, 14, 20, 5, 9, 14, 20,
    4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23,
    6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21
]

SIN_CONSTANTS = [
    int(abs(math.sin(i + 1)) * 2**32) & 0xFFFFFFFF for i in range(64)
]

INITIAL_HASH_VALUES = [0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476]

MD5_FUNCTIONS = (
    16 * [lambda b, c, d: (b & c) | (~b & d)] +
    16 * [lambda b, c, d: (d & b) | (~d & c)] +
    16 * [lambda b, c, d: b ^ c ^ d] +
    16 * [lambda b, c, d: c ^ (b | ~d)]
)

INDEX_TRANSFORM_FUNCTIONS = (
    16 * [lambda i: i] +
    16 * [lambda i: (5 * i + 1) % 16] +
    16 * [lambda i: (3 * i + 5) % 16] +
    16 * [lambda i: (7 * i) % 16]
)

def left_rotate(value, shift_amount):
    value &= 0xFFFFFFFF
    return ((value << shift_amount) | (value >> (32 - shift_amount))) & 0xFFFFFFFF

In [2]:
def md5(message: str) -> str:
    message = bytearray(message.encode())
    original_length_bits = (8 * len(message)) & 0xFFFFFFFFFFFFFFFF
    message.append(0x80)
    while len(message) % 64 != 56:
        message.append(0)
    message += original_length_bits.to_bytes(8, byteorder='little')

    hash_values = INITIAL_HASH_VALUES[:]
    for chunk_start in range(0, len(message), 64):
        a, b, c, d = hash_values
        chunk = message[chunk_start:chunk_start + 64]

        for i in range(64):
            f = MD5_FUNCTIONS[i](b, c, d)
            g = INDEX_TRANSFORM_FUNCTIONS[i](i)
            to_rotate = (
                a + f + SIN_CONSTANTS[i] +
                int.from_bytes(chunk[4 * g:4 * g + 4], byteorder='little')
            )
            new_b = (b + left_rotate(to_rotate, ROTATION_AMOUNTS[i])) & 0xFFFFFFFF
            a, b, c, d = d, new_b, b, c

        for i, value in enumerate([a, b, c, d]):
            hash_values[i] += value
            hash_values[i] &= 0xFFFFFFFF

    s = sum(x << (32 * i) for i, x in enumerate(hash_values))
    raw_bytes = s.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw_bytes, byteorder='big'))


In [3]:
import hashlib

def check(msg: str, hashsum: str) -> bool:
    return hashlib.md5(msg.encode()).hexdigest() == hashsum

msg = "FBIT 336959"
md5_hash = md5(msg)
print(md5_hash)
print(check(msg, md5_hash))


3e9afbc27864a4d0b2d425247128e600
True
